## Loading Dependencies

In [3]:
import pandas as pd


In [4]:
# path to the folder containing movielens data
Path = "D:\OneDrive - University of Windsor\Danial Ebrat _ PhD projects\Comprehensive Exam\Recommender-Research-Baseline (1)\Movielens1m\ml-1m"

## Loading users information and creating user_info column

In [5]:
# Loading the user data
user_df = pd.read_csv(f'{Path}/users.dat', 
                      sep='::', 
                      names=["user_id", "gender", "age", "occupation", "zip_code"], 
                      encoding='latin-1')

user_df.head(100)

C:\Users\dania\AppData\Local\Temp\ipykernel_4076\3895213713.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  user_df = pd.read_csv(f'{Path}/users.dat',


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\OneDrive - University of Windsor\\Danial Ebrat _ PhD projects\\Comprehensive Exam\\Recommender-Research-Baseline (1)\\Movielens100K\\ml-100k/users.dat'

In [68]:
# Define the mapping dictionaries
age_dict = {
    1: "Under 18",
    18: "18-24",
    25: "25-34",
    35: "35-44",
    45: "45-49",
    50: "50-55",
    56: "56+"
}

occupation_dict = {
    0:  "other or not specified",
    1:  "academic/educator",
    2:  "artist",
    3:  "clerical/admin",
    4:  "college/grad student",
    5:  "customer service",
    6:  "doctor/health care",
    7:  "executive/managerial",
    8:  "farmer",
    9:  "homemaker",
    10: "K-12 student",
    11: "lawyer",
    12: "programmer",
    13: "retired",
    14: "sales/marketing",
    15: "scientist",
    16: "self-employed",
    17: "technician/engineer",
    18: "tradesman/craftsman",
    19: "unemployed",
    20: "writer"
}

gender_dict = {
    'M': 'Male',
    'F': 'Female'
}

# Map the codes to the actual values
user_df['age'] = user_df['age'].map(age_dict)
user_df['occupation'] = user_df['occupation'].map(occupation_dict)
user_df['gender'] = user_df['gender'].map(gender_dict)


# loading Movies
user_df.head(100)

,user_id,gender,age,occupation,zip_code
0,1,Female,Under 18,K-12 student,48067
1,2,Male,56+,self-employed,70072
2,3,Male,25-34,scientist,55117
3,4,Male,45-49,executive/managerial,02460
4,5,Male,25-34,writer,55455
...,...,...,...,...,...
95,96,Female,25-34,self-employed,78028
96,97,Female,35-44,clerical/admin,66210
97,98,Female,35-44,executive/managerial,33547
98,99,Female,Under 18,K-12 student,19390


In [69]:

# Function to format the user info
def format_user_info(row):
    # Create the formatted string
    user_info = f"""
    Age: {row['age']}
    Gender: {row['gender']}
    Occupation: {row['occupation']}
    """
    return user_info

# Apply the function to create the 'user_info' column
user_df['user_info'] = user_df.apply(format_user_info, axis=1)

user_df.head()


,user_id,gender,age,occupation,zip_code,user_info
0,1,Female,Under 18,K-12 student,48067,\n Age: Under 18\n Gender: Female\n O...
1,2,Male,56+,self-employed,70072,\n Age: 56+\n Gender: Male\n Occupati...
2,3,Male,25-34,scientist,55117,\n Age: 25-34\n Gender: Male\n Occupa...
3,4,Male,45-49,executive/managerial,02460,\n Age: 45-49\n Gender: Male\n Occupa...
4,5,Male,25-34,writer,55455,\n Age: 25-34\n Gender: Male\n Occupa...


In [70]:
user_df.to_csv("users_with_summary_df.csv")
user_df.to_pickle("users_with_summary_df.pkl")

## Loading movies information and creating movie_info column: 

In [46]:
item_df = pd.read_csv(f'{Path}/movies.dat', sep='::', names=["MovieID", "Title", "Genres"], encoding='latin-1')

item_df.head()

C:\Users\dania\AppData\Local\Temp\ipykernel_13992\1191591061.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  item_df = pd.read_csv(f'{Path}/movies.dat', sep='::', names=["MovieID", "Title", "Genres"], encoding='latin-1')


,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [47]:

print(len(item_df))

3883


## Using TMDB API to extract more information about the movies

In [48]:
import requests
import pandas as pd
import time
import re
from tqdm import tqdm


API_KEY = "your api key"

def get_movie_details(title, api_key):
    """Fetch movie details from TMDb API given a movie title, potentially including a year."""
    # Extract the year and clean title if present in the format "Title (Year)"
    match = re.match(r"^(.*?)\s*\((\d{4})\)$", title)
    if match:
        clean_title, year = match.groups()
    else:
        clean_title, year = title, None
    
    try:
        # Modify the URL to include the year in the search if it's available
        url = f"https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={clean_title}"
        if year:
            url += f"&year={year}"
        response = requests.get(url)
        data = response.json()
        if data['results']:
            movie_id = data['results'][0]['id']  # Assuming the first result is the correct movie
            details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}"
            details_response = requests.get(details_url)
            return details_response.json()  # Returns a dictionary of details
        else:
            return {}
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        time.sleep(10)  # Wait for 10 seconds before retrying
        return get_movie_details(title, api_key)


# # Using tqdm for a progress bar
item_df['details'] = [get_movie_details(title, API_KEY) for title in tqdm(item_df['Title'], desc="Fetching movie details")]

# Expand 'details' dict into separate columns, excluding the duplicate 'title'
details_df = item_df['details'].apply(pd.Series)

# Drop the "title" column from details_df if it exists
details_df.drop(columns=['Title'], errors='ignore', inplace=True)

# Concatenate the original DataFrame and the expanded details DataFrame
item_df = pd.concat([item_df.drop(['details'], axis=1), details_df], axis=1)


Fetching movie details: 100%|██████████| 3883/3883 [14:39<00:00,  4.42it/s]  


In [ ]:
# Another version -- more efficeint

import requests
import pandas as pd
import time
import re
from tqdm import tqdm
from requests.adapters import HTTPAdapter, Retry
from functools import lru_cache
import threading
import os

API_KEY = "your api key"

# Rate limiting: 40 requests per 10 seconds
RATE_LIMIT = 40
RATE_PERIOD = 10  # seconds
request_counts = []
lock = threading.Lock()

# Create a session with a retry strategy
session = requests.Session()
retries = Retry(total=5,
                backoff_factor=1,
                status_forcelist=[429, 500, 502, 503, 504],
                method_whitelist=["HEAD", "GET", "OPTIONS"])
adapter = HTTPAdapter(max_retries=retries)
session.mount('https://', adapter)
session.mount('http://', adapter)

def rate_limited_request(method, url, **kwargs):
    """Perform a rate-limited request."""
    with lock:
        current_time = time.time()
        # Remove outdated request timestamps
        while request_counts and request_counts[0] <= current_time - RATE_PERIOD:
            request_counts.pop(0)
        if len(request_counts) >= RATE_LIMIT:
            sleep_time = RATE_PERIOD - (current_time - request_counts[0])
            time.sleep(sleep_time)
        request_counts.append(time.time())
    response = session.request(method, url, **kwargs)
    return response

@lru_cache(maxsize=None)
def get_movie_details(title, api_key):
    """Fetch movie details from TMDb API given a movie title, potentially including a year."""
    # Extract the year and clean title if present in the format "Title (Year)"
    match = re.match(r"^(.*?)\s*\((\d{4})\)$", title)
    if match:
        clean_title, year = match.groups()
    else:
        clean_title, year = title, None

    # Encode the title for URL
    clean_title_encoded = requests.utils.quote(clean_title)

    # Build the search URL
    url = f"https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={clean_title_encoded}"
    if year:
        url += f"&year={year}"

    try:
        response = rate_limited_request('GET', url, timeout=10)
        response.raise_for_status()
        data = response.json()
        if data.get('results'):
            movie_id = data['results'][0]['id']  # Assuming the first result is correct
            details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}"
            details_response = rate_limited_request('GET', details_url, timeout=10)
            details_response.raise_for_status()
            return details_response.json()  # Returns a dictionary of details
        else:
            return {}
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred for '{title}': {http_err}")
        return {}
    except requests.exceptions.RequestException as req_err:
        print(f"Request exception for '{title}': {req_err}")
        return {}
    except Exception as e:
        print(f"An error occurred for '{title}': {e}")
        return {}

# Ensure the 'Title' column exists
if 'Title' not in item_df.columns:
    raise KeyError("The DataFrame does not contain a 'Title' column.")

# Apply the function to the DataFrame with a progress bar
tqdm.pandas(desc="Fetching movie details")
item_df['details'] = item_df['Title'].progress_apply(lambda x: get_movie_details(x, API_KEY))

# Expand 'details' dict into separate columns
details_df = pd.json_normalize(item_df['details'])

# Concatenate the original DataFrame and the expanded details DataFrame
item_df = pd.concat([item_df.drop(['details'], axis=1), details_df], axis=1)


In [50]:
item_df.head()

item_df.to_csv("movies_1m_tmdb.csv")

In [51]:
# Filter out rows where the 'overview' column is NaN
cleaned_df = item_df.dropna(subset=['overview'])

# Saving the cleaned DataFrame
cleaned_df.to_csv('movies_enriched_dataset.csv', index=False)

print("Cleaned dataset saved. Number of movies with overviews:", cleaned_df.shape[0])

Cleaned dataset saved. Number of movies with overviews: 3641


In [76]:


cleaned_df.rename(columns={"MovieID": "movie_id"}, inplace=True)


C:\Users\dania\AppData\Local\Temp\ipykernel_13992\566873030.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df.rename(columns={"MovieID": "movie_id"}, inplace=True)


In [77]:
cleaned_df.head()

,movie_id,Title,Genres,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,movie_info
0,1,Toy Story (1995),Animation|Children's|Comedy,False,/lxD5ak7BOoinRNehOCA85CQ8ubr.jpg,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 12, '...",http://toystory.disney.com/toy-story,862.0,...,394436586.0,81.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Hang on for the comedy that goes to infinity a...,Toy Story,False,7.970,18309.0,\n Movie title: Toy Story (1995) \n Genr...
1,2,Jumanji (1995),Adventure|Children's|Fantasy,False,/i7jKGJZjGmLZFt48xLtwSgn9Cdw.jpg,"{'id': 495527, 'name': 'Jumanji Collection', '...",65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://www.sonypictures.com/movies/jumanji/,8844.0,...,262821940.0,104.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Roll the dice and unleash the excitement!,Jumanji,False,7.200,10462.0,\n Movie title: Jumanji (1995) \n Genres...
2,3,Grumpier Old Men (1995),Comedy|Romance,False,/nh9gYaXHTNT9yylX10L9aGqFehy.jpg,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",25000000.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",,15602.0,...,71500000.0,101.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.500,375.0,\n Movie title: Grumpier Old Men (1995) \n ...
3,4,Waiting to Exhale (1995),Comedy|Drama,False,/jZjoEKXMTDoZAGdkjhAdJaKtXSN.jpg,None,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",,31357.0,...,81452156.0,127.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.300,162.0,\n Movie title: Waiting to Exhale (1995) \n...
4,5,Father of the Bride Part II (1995),Comedy,False,/lEsjVrGU21BeJjF5AF9EWsihDpw.jpg,"{'id': 96871, 'name': 'Father of the Bride (St...",0.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10751, '...",,11862.0,...,76594107.0,106.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Just when his world is back to normal... he's ...,Father of the Bride Part II,False,6.237,725.0,\n Movie title: Father of the Bride Part II...


In [78]:
cleaned_df.to_pickle('movies_enriched_dataset.pkl')

## Loading enriched dataset and creating movie_info col

In [54]:

# Function to extract and format the genres
def format_movie_info(row):

    movie_info = f"""
    Movie title: {row['Title']} 
    Genres: {row['Genres']}
    Overview: {row['overview']} 
    """
    return movie_info

# Apply the function to create the 'movie_info' column
cleaned_df['movie_info'] = cleaned_df.apply(format_movie_info, axis=1)
cleaned_df.head()

C:\Users\dania\AppData\Local\Temp\ipykernel_13992\2752981810.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['movie_info'] = cleaned_df.apply(format_movie_info, axis=1)


,MovieID,Title,Genres,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,movie_info
0,1,Toy Story (1995),Animation|Children's|Comedy,False,/lxD5ak7BOoinRNehOCA85CQ8ubr.jpg,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 12, '...",http://toystory.disney.com/toy-story,862.0,...,394436586.0,81.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Hang on for the comedy that goes to infinity a...,Toy Story,False,7.970,18309.0,\n Movie title: Toy Story (1995) \n Genr...
1,2,Jumanji (1995),Adventure|Children's|Fantasy,False,/i7jKGJZjGmLZFt48xLtwSgn9Cdw.jpg,"{'id': 495527, 'name': 'Jumanji Collection', '...",65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://www.sonypictures.com/movies/jumanji/,8844.0,...,262821940.0,104.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Roll the dice and unleash the excitement!,Jumanji,False,7.200,10462.0,\n Movie title: Jumanji (1995) \n Genres...
2,3,Grumpier Old Men (1995),Comedy|Romance,False,/nh9gYaXHTNT9yylX10L9aGqFehy.jpg,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",25000000.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",,15602.0,...,71500000.0,101.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.500,375.0,\n Movie title: Grumpier Old Men (1995) \n ...
3,4,Waiting to Exhale (1995),Comedy|Drama,False,/jZjoEKXMTDoZAGdkjhAdJaKtXSN.jpg,None,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",,31357.0,...,81452156.0,127.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.300,162.0,\n Movie title: Waiting to Exhale (1995) \n...
4,5,Father of the Bride Part II (1995),Comedy,False,/lEsjVrGU21BeJjF5AF9EWsihDpw.jpg,"{'id': 96871, 'name': 'Father of the Bride (St...",0.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10751, '...",,11862.0,...,76594107.0,106.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Just when his world is back to normal... he's ...,Father of the Bride Part II,False,6.237,725.0,\n Movie title: Father of the Bride Part II...


In [55]:

# Saving the cleaned DataFrame

cleaned_df.to_pickle('movies_enriched_dataset.pkl')
# user_df.to_pickle('user_dataset.pkl')



In [57]:
movies_without_overview = item_df[item_df['overview'].isna()]

movies_without_overview.shape

(242, 29)

## Loading ratings information

In [84]:
# Load the ratings data
rating_df = pd.read_csv(f'{Path}/u1.base', 
                        sep='\t', 
                        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
                        encoding='latin-1')
rating_df.head()

,user_id,movie_id,rating,timestamp
0,1229,34,4,974836942
1,1229,50,4,974836806
2,1229,80,3,974836917
3,1229,162,5,974836917
4,1229,306,3,974836843


In [86]:
# Sort by user_id and timestamp descending
rating_df.sort_values(by=['user_id', 'timestamp'], ascending=[True, False], inplace=True)

# Group by user_id and take the first 60 entries for each user
filtered_df = rating_df.groupby('user_id').head(40)

# Calculate the percentage of the training set used
percentage_used = (len(filtered_df) / len(rating_df)) * 100

# Print the result
print(f"Percentage of training set used: {percentage_used:.2f}%")

Percentage of training set used: 22.56%


In [87]:
rating_df.groupby('user_id').count().sort_index(ascending=True)

,movie_id,rating,timestamp
user_id,,,
1229,14,14,14
1230,253,253,253
1231,110,110,110
1232,135,135,135
1233,61,61,61
...,...,...,...
6036,888,888,888
6037,202,202,202
6038,20,20,20


In [61]:
rating_df[rating_df['user_id'] == 1].head(100)

,user_id,movie_id,rating,timestamp


In [63]:
rating_test_df = pd.read_csv(f'{Path}/u1.test', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'], encoding='latin-1')
rating_test_df.head()

,user_id,movie_id,rating,timestamp
0,1,1,5,978824268
1,1,48,5,978824351
2,1,150,5,978301777
3,1,260,4,978300760
4,1,527,5,978824195


In [64]:
rating_test_df.groupby('user_id').count()

,movie_id,rating,timestamp
user_id,,,
1,53,53,53
2,129,129,129
3,51,51,51
4,21,21,21
5,198,198,198
...,...,...,...
1224,578,578,578
1225,28,28,28
1226,21,21,21


In [47]:
rating_test_df[rating_df['user_id'] == 1].head(100)

C:\Users\dania\AppData\Local\Temp\ipykernel_52524\2948317273.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rating_test_df[rating_df['user_id'] == 1].head(100)


,user_id,movie_id,rating,timestamp
0,1,6,5,887431973
1,1,10,3,875693118
2,1,12,5,878542960
3,1,14,5,874965706
4,1,17,3,875073198
...,...,...,...,...
95,1,202,5,875072442
96,1,206,4,876893205
97,1,208,5,878542960
98,1,209,4,888732908


In [3]:
rating_train_df = pd.read_csv(f'{Path}/u1.base', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'], encoding='latin-1')
rating_train_df.head()

,user_id,movie_id,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [5]:
rating_train_df.groupby('user_id').count()

,movie_id,rating,timestamp
user_id,,,
1,135,135,135
2,40,40,40
3,28,28,28
4,14,14,14
5,91,91,91
...,...,...,...
939,49,49,49
940,107,107,107
941,22,22,22


In [7]:
# Load movies
movies_df = pd.read_pickle("./Data/movies_enriched_dataset.pkl")
movies_df = movies_df[["movie_id", "movie_info"]]

In [8]:
movies_df.head(300)

,movie_id,movie_info
0,1,\n Movie title: Toy Story (1995) \n Genr...
1,2,\n Movie title: GoldenEye (1995) \n Genr...
2,3,\n Movie title: Four Rooms (1995) \n Gen...
3,4,\n Movie title: Get Shorty (1995) \n Gen...
4,5,\n Movie title: Copycat (1995) \n Genre:...
...,...,...
304,305,"\n Movie title: Ice Storm, The (1997) \n ..."
305,306,"\n Movie title: Mrs. Brown (Her Majesty, Mr..."
306,307,"\n Movie title: Devil's Advocate, The (1997..."
307,308,\n Movie title: FairyTale: A True Story (19...


In [12]:
print(len(movies_df))

1591


In [16]:
processed_ratings_file = "./Data/rating_test_df.pkl"
rating_test_df = pd.read_pickle(processed_ratings_file)

cleaned_df = rating_test_df.dropna(subset=['simulated_ratings'])
cleaned_df.head(100)

,user_id,movie_id,rating,timestamp,simulated_ratings
137,2,13,4,888551922,4
138,2,19,3,888550871,5
139,2,50,5,888552084,4
140,2,251,5,888552084,4
141,2,257,4,888551062,4
144,2,281,3,888980240,3
145,2,290,3,888551441,3
146,2,292,4,888550774,4
147,2,297,4,888550871,5
148,2,298,3,888551441,4


In [7]:
movies_df = pd.read_pickle("./Data/movies_enriched_dataset.pkl")
movies_df = movies_df[["movie_id", "movie_info"]]

In [8]:
movies_df.head()

,movie_id,movie_info
0,1,\n Movie title: Toy Story (1995) \n Genr...
1,2,\n Movie title: GoldenEye (1995) \n Genr...
2,3,\n Movie title: Four Rooms (1995) \n Gen...
3,4,\n Movie title: Get Shorty (1995) \n Gen...
4,5,\n Movie title: Copycat (1995) \n Genre:...
